In [2]:
!pip install dask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn import decomposition, preprocessing, cluster, tree
import dask.dataframe as dd

In [5]:
peptide_file = "PD-datasets/train_peptides.csv"
peptides = dd.read_csv(peptide_file)
peptides

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
npartitions=1,,,,,,
,object,int64,int64,object,object,float64
,...,...,...,...,...,...


In [6]:
proteins_file = "PD-datasets/train_proteins.csv"
proteins = dd.read_csv(proteins_file)
proteins

,visit_id,visit_month,patient_id,UniProt,NPX
npartitions=1,,,,,
,object,int64,int64,object,float64
,...,...,...,...,...


In [4]:
clinical_file = "PD-datasets/train_clinical_data.csv"
clinical = pd.read_csv(clinical_file)
clinical_df = pd.DataFrame(clinical)
clinical_df.set_index("visit_id",inplace=True)
clinical_df.head()

,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
visit_id,,,,,,,
55_0,55,0,10.0,6.0,15.0,NaN,NaN
55_3,55,3,10.0,7.0,25.0,NaN,NaN
55_6,55,6,8.0,10.0,34.0,NaN,NaN
55_9,55,9,8.0,9.0,30.0,0.0,On
55_12,55,12,10.0,10.0,41.0,0.0,On


In [5]:
# merge all of the datasets together
protpep = pd.merge(left=peptides_df,right=proteins_df,how='outer',on="visit_id")

In [6]:
protpep = protpep.drop(columns=["visit_month_y", "patient_id_y"])

In [7]:
protpep.columns

Index(['visit_id', 'visit_month_x', 'patient_id_x', 'UniProt_x', 'Peptide',
       'PeptideAbundance', 'UniProt_y', 'NPX'],
      dtype='object')

In [8]:
protpep.set_index("visit_id",inplace=True)

In [9]:
merged_df = protpep.join(clinical_df)

In [10]:
#take a look at the data and identify what needs to be done to prepare for ML
merged_df.head(25)

,visit_month_x,patient_id_x,UniProt_x,Peptide,PeptideAbundance,UniProt_y,NPX,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
visit_id,,,,,,,,,,,,,,
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O00391,9104.27,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O00533,402321.00,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O14773,7150.57,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O14791,2497.84,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O15240,83002.90,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O15394,15113.60,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O43505,167327.00,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O60888,129048.00,10053.0,0.0,3.0,0.0,13.0,0.0,NaN
10053_0,0,10053,O00391,NEQEQPLGQWHLS,9104.27,O75144,53069.50,10053.0,0.0,3.0,0.0,13.0,0.0,NaN


In [ ]:
cleaned_df = merged_df.drop(columns=["visit_month","patient_id"])

In [ ]:
cleaned_df.dtypes

In [ ]:
pd.get_dummies(merged_df, columns=["",""])

In [ ]:
# PCA

X = #cleaned data will go here
std = preprocessing.StandardScaler()
X_std = pd.DataFrame(std.fit_transform(X), columns=X.columns)
X_std

In [ ]:
pca = decomposition.PCA()
pca_X = pd.DataFrame(pca.fit_transform(X_std), columns=[f'PC{i+1}' for i in range(len(X.columns))])
pca_X

In [ ]:
# plot PCs and dendrogram